In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train=pd.read_csv("/kaggle/input/train1.csv")
test=pd.read_csv("/kaggle/input/test1.csv")
print(train.shape,test.shape)

In [ ]:
import numpy as np
import pandas as pd

#PLOTS
import seaborn as sns
import matplotlib.pyplot as plt

# STATS
from scipy.stats import skew,norm

# Ignore useless warnings
import warnings
warnings.filterwarnings(action="ignore")

In [ ]:
## EDA
train.head()

In [ ]:
#Sales Price stats
ax = sns.distplot(train['SalePrice'],fit=norm)

In [ ]:
#sales price is skewed to the right.
train["SalePrice"] = np.log1p(train["SalePrice"])

#Let's again plot it
ax = sns.distplot(train['SalePrice'],fit=norm)

In [ ]:
#skew and kurt
price=train["SalePrice"]
print(" Kurtosis:",price.kurt(),"\n","Skewness:" ,price.skew())

In [ ]:
#Correlations among variables
corr=train.corr()
plt.figure(figsize=(40,30))
sns.heatmap(corr,annot=True,cmap='Blues')

In [ ]:
##saleprice correlation matrix                   //original site-https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python#Out-liars!
k =10 #number of variables for heatmap           //cell-12
cols = corr.nlargest(k, 'SalePrice')
cols=cols['SalePrice']
cols_index=cols.index
# print(cols_index)
# p=train[cols_index].values
# print(p)
# print(p.shape)
cm = np.corrcoef(train[cols_index].values.T) 
# cm=pd.DataFrame(cm)
# print(cm)
# print(cm.shape)
plt.figure(figsize=(20,15))
sns.set(font_scale=1.25)
hm = sns.heatmap(cm,cmap='Blues', cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols_index, xticklabels=cols_index) 
plt.show()

In [ ]:
# Pairplots of highly correlated variables with sales price
cols=['SalePrice','OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea',
       'TotalBsmtSF', '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt']
sns.pairplot(train[cols],size=1.5)
plt.show()

In [ ]:
# Box plots 
ax = sns.boxplot(x="OverallQual", y="SalePrice", data=train)

In [ ]:
plt.figure(figsize=(20,15))
plt.xticks(rotation=90)
ax = sns.boxplot(x="YearBuilt", y="SalePrice", data=train)

FEATURE ENGINEERING

Removing outliers

In [ ]:
ax = sns.boxplot(y="SalePrice", data=train,orient='v')

In [ ]:
train.drop(train[train['SalePrice']>300000].index,inplace=True)
ax = sns.boxplot(y="SalePrice", data=train,orient='v')
train.shape

In [ ]:
train_labels = train['SalePrice'].reset_index(drop=True)
train_features = train.drop(['SalePrice'], axis=1)
test_features = test

# Combine train and test features in order to apply the feature transformation to the entire dataset
all_features = pd.concat([train_features, test_features]).reset_index(drop=True)
all_features.shape

Handling missing values

In [ ]:
total_missing=(all_features.isnull().sum()).sort_values(ascending=False)
missing_percent=(all_features.isnull().mean()*100).sort_values(ascending=False)
missing_data = pd.concat([total_missing, missing_percent],axis=1,keys=['total_missing','missing_percent'])
missing_data.head(35)

In [ ]:
all_features.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu','GarageCond','GarageFinish','GarageQual','GarageType','BsmtCond','BsmtQual','BsmtExposure','BsmtFinType1','BsmtFinType2'], axis=1,inplace=True)
total_missing=(all_features.isnull().sum()).sort_values(ascending=False)
missing_percent=(all_features.isnull().mean()*100).sort_values(ascending=False)
missing_data = pd.concat([total_missing, missing_percent],axis=1,keys=['total_missing','missing_percent'])
missing_data.head(20)

In [ ]:
all_features=all_features.fillna(train.mean()) 
total_missing=(all_features.isnull().sum()).sort_values(ascending=False)
missing_percent=(all_features.isnull().mean()*100).sort_values(ascending=False)
missing_data = pd.concat([total_missing, missing_percent],axis=1,keys=['total_missing','missing_percent'])
missing_data.head(20)

In [ ]:
#replacing with its mode
all_features['Electrical']=all_features['Electrical'].fillna("SBrkr")   
all_features['MasVnrType']=all_features['MasVnrType'].fillna("None")

#'GarageX' variables have the same number of missing data. I bet missing data refers to the same set of observations.
#Since the most important information regarding garages is expressed by 'GarageCars' and considering that we are just talking about 5% of missing data,
#I'll delete the mentioned 'GarageX' variables. The same logic applies to 'BsmtX' variables.
all_features.dropna(axis=0,inplace=True)
all_features.reset_index(drop=True,inplace=True)

In [ ]:
#checking if still there is null values or not
total_missing=(all_features.isnull().sum()).sort_values(ascending=False)
missing_percent=(all_features.isnull().mean()*100).sort_values(ascending=False)
missing_data = pd.concat([total_missing, missing_percent],axis=1,keys=['total_missing','missing_percent'])
missing_data.head(20)
all_features

Handling skewed features

In [ ]:
#getting all numerical features for calculation of skewness and boxplot
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric = []
for i in all_features.columns:
    if all_features[i].dtype in numeric_dtypes:
        numeric.append(i)

In [ ]:
#Box plt for numeric features
f, ax = plt.subplots(figsize=(10, 12))
ax.set_xscale("log")
ax = sns.boxplot(data=all_features[numeric] , orient="h")

In [ ]:
skew_features = all_features[numeric].apply(lambda x: skew(x)).sort_values(ascending=False)
print(numeric)
skew_features.head(30)  #features having skewness>0.5 are high skewed

In [ ]:
#Feature Scaling(Normalising skewed features)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_features=scaler.fit_transform(all_features[numeric].values)
scaled_features=pd.DataFrame(scaled_features)
scaled_features.columns=['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
all_features=all_features.drop(columns=numeric)
all_features=pd.concat([all_features, scaled_features],axis=1).reset_index(drop=True)
all_features

In [ ]:
#checking fautures after normalisation
f, ax = plt.subplots(figsize=(10, 12))
ax.set_xscale("log")
ax = sns.boxplot(data= all_features , orient="h")

In [ ]:
#checking skewness after normalisation
skew_features = all_features[numeric].apply(lambda x: skew(x)).sort_values(ascending=False)
skew_features #skewness dont change after normalisation

In [ ]:
#Encode categorical features
all_features = pd.get_dummies(all_features).reset_index(drop=True)
all_features

In [ ]:
all_features.head()

**Recreating training and test set**

In [ ]:
train=all_features.iloc[:len(train_labels), :]
test=all_features.iloc[len(train_labels): , :]
train.shape,test.shape

**Training different models**

In [ ]:
# Setup cross validation folds
from sklearn.model_selection import KFold
kf = KFold(n_splits=12, random_state=42, shuffle=True)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
model=LinearRegression()
rmse=np.sqrt(-cross_val_score(model, train, train_labels, scoring="neg_mean_squared_error", cv=kf))
print(np.mean(rmse))

In [ ]:
regression_model=LinearRegression()
regression_model.fit(train,train_labels)
y_predicted = regression_model.predict(test)
y_predicted=pd.DataFrame(y_predicted)
y_predicted.columns=['SalePrice']
# y_predicted = y_predicted.reset_index()
# y_predicted.columns[1] = 'ID'
# y_predicted['ID'] = y_predicted.index + 1460
# y_predicted
df=y_predicted
df.insert(0, 'ID', range(1461, 1461 + len(df)))
df

In [ ]:
# XGBoost Regressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from xgboost import XGBRegressor
xgboost = XGBRegressor(learning_rate=0.01,
                       n_estimators=6000,
                       max_depth=4,
                       min_child_weight=0,
                       gamma=0.6,
                       subsample=0.7,
                       colsample_bytree=0.7,
                       objective='reg:squarederror',
                       nthread=-1,
                       scale_pos_weight=1,
                       seed=27,
                       reg_alpha=0.00006,
                       random_state=42)
model=xgboost
rmse = np.sqrt(-cross_val_score(model, train, train_labels, scoring="neg_mean_squared_error", cv=kf))
np.mean(rmse)

In [ ]:
#can do with xgboost too but not done...learnt the concept behind it

In [ ]:
# Read in sample_submission dataframe
submission = pd.read_csv("/kaggle/input/sample_submission1.csv")
submission.shape

In [ ]:
# Append predictions from blended models
submission.iloc[:,1] = np.floor(np.expm1(y_predicted['SalePrice']))  #submitted after changing log(1+saleprice) to saleprice
submission.to_csv("submission_regression1.csv", index=False)